In [24]:
""" Imports """
import json
import pandas as pd
import polars as pl

# Defining a Database Schema

## What data do we need

Starting from the top, what information should we display to users?

example return field in csv format (use tsv since a few course/dept names have commas)

Example: user selects inst=UCSC, course=MATH19A
`San Diego Miramar College, MATH 150, Calculus with Analytic Geometry I, 4 units`

Example: user selects inst=UCSD, course=CHEM6B

`San Diego Miramar College, CHEM 200, General Chemistry I - Lecture, 3 units`

`and`

`San Diego Miramar College, CHEM 201, General Chemistry II - Lecture, 3 units`


Required data:
- sending institution (e.g. San Diego Miramar College)
- course prefix (e.g. CHEM)
- course number (e.g. 201)
- course name (e.g. General Chemistry II - Lecture)
- unit count (e.g 3 units)
- course ID (some string of numbers, makes mappings easy to deal with)


## The schema

Table 1: course glossary

fields:
- id (int, primary key)  // course id
- inst (string)  // community college or univ
- prefix (string)  // course prefix (e.g. CHEM)
- course number (int) 
- course name (string)
- min units (int)  // if min units and max units are the same, display 1
- max units (int)

Table 2: articulations
fields:
- id: (int, primary key)
- inst (uni) (string)
- agreements (json string)

## The JSON string

tbd, working on that rn

## The query flow

- user enters site
- user picks university -> get list of all courses at university for dropdown
    - sends request to backend
    - backend uses user input to send query to db
        -  `FROM glossary SELECT * WHERE inst IS {whatever the user picked}`
    - backend converts results into json w/ id: {prefix, num, name, min units, max units}  // use for formatting dropdown
    - backend returns json
- user picks course from list -> get list of all articulated courses from id
    - sends request to backend
    - backend uses user input to send query to db
        - `FROM articulations SELECT agreements WHERE id IS {id of course user picked}`
        - returns 1 json string with all articulations
    - backend aggregates all json string'd course IDs into list, queries course data
        - `FROM glossary SELECT * WHERE id IS {ids in list}`
    - backend converts results into json w/ id: {inst, prefix, num, name, min units, max units}  // actual displayed data
    - backend returns both jsons (articulation string w/ IDs and the cc glossary)
- both jsons formatted into cells on frontend
- user gets results

In [25]:
""" Define functions for generating an in-memory glossary of every course in an AllPrefixes agreement page """

def get_query(cc_id: int, uni_id: int) -> list[dict]:
    with open(f"./data/{uni_id}/{cc_id}to{uni_id}.json", "r") as fp:
        out = json.load(fp)
    return out
    

def update_courses(courselist: list[dict], glossary: dict, inst: int) -> None:
    for course in courselist:
        if not all((course["prefix"],
                    course["courseNumber"],
                    course["courseTitle"],
                    course["minUnits"],
                    course["maxUnits"],
                    course["begin"])):
            continue
            
        course_id: int = course["courseIdentifierParentId"]
        
        if course_id not in glossary:            
            glossary[course_id] = {
                "course_id": course_id,
                "inst_id": int(inst),
                "course_code": f"{course["prefix"]} {course["courseNumber"]}",
                "course_name": course["courseTitle"],
                "min_units": int(course["minUnits"]),
                "max_units": int(course["maxUnits"]),
                "begin": course["begin"]
            }


def create_course_glossary(cc: int, uni: int) -> pl.DataFrame:
    # create output glossary
    course_glossary = dict()
    
    with open(f"./data/{uni}/{cc}to{uni}.json", "r") as fp:
        articulations = json.load(fp)
    
    # populate query
    for dept in articulations:
        articulationList = dept["articulations"] if "articulations" in dept else [dept["articulation"]]
        
        
        for articulation in articulationList:
            if "requirement" in articulation and "course" not in articulation and "series" not in articulation:
                continue
            # handle university courses
            uni_courses: list[dict] = [articulation["course"]] if "course" in articulation else articulation["series"]["courses"]
                
            update_courses(courselist=uni_courses, glossary=course_glossary, inst=uni)
                
            # handle cc courses
            agreements: dict | None = articulation["sendingArticulation"]
            if isinstance(agreements, dict):
                for agreement in agreements["items"]:
                    if "items" in agreement:
                        update_courses(courselist=agreement["items"], glossary=course_glossary, inst=cc)
                    else:
                        with open("./data/known_errors.tsv", "a") as fp:
                            fp.write(f"{cc}\t{uni}\tKnown issue with parsing 'Select 1 course from following' as OR block\n")
        
    return pl.DataFrame(pd.DataFrame(course_glossary.values(), index=course_glossary.keys()))


"""
Articulations are expected to have duplicates
course_id : agreement json -> for 1 uni:cc pairing, therefore many of these should exist in the final table
queries will look like: SELECT articulation FROM articulations WHERE 'course_id' == [number]
which will return a slice
"""
def extract_articulations(cc: int, uni: int) -> pl.DataFrame:
    # extract data from query & construct polars dataframe
    articulations: pl.DataFrame = pl.from_pandas(pd.DataFrame(get_query(cc, uni)))
    
    colname = "articulation"
    if "articulations" in articulations.columns:
        articulations = articulations.explode("articulations")
        colname = "articulations"
    articulations_struct = articulations.get_column(colname).struct

    articulations = pl.concat((
        articulations_struct.field("course").struct.field("courseIdentifierParentId").rename("course_id").to_frame(),
        articulations_struct.field("series").to_frame(),
        articulations_struct.field("sendingArticulation").to_frame()
    ), how="horizontal")
    
    articulations = articulations.with_columns(
        pl.lit(cc).alias("cc"),
        pl.lit(uni).alias("uni")
    )
    
    
    # transform into mapping of course id : relationship to course ids that articulate to it
    articulations = (
        articulations
        
        # extract courses from uni series objects and treat them as individual courses
        # premise: A and B articulates to C and D => A and B articulates to C and A and B articulates D
        .with_columns(pl.col("series").struct.field("courses"))
        .explode("courses")
        
        # extract sendingArticulation field (contains A and B) and id of C/D courses
        .with_columns(
            course_id=pl.coalesce(
                "course_id",
                pl.col("courses").struct.field("courseIdentifierParentId")
            ),
            items=(
                pl.when(pl.col("sendingArticulation").struct.field("items").list.len() > 0)
                .then(pl.col("sendingArticulation").struct.field("items"))
                .otherwise(None)
            )
        )
        
        # transform sendingArticulation to only keep course ids from whole course structs for C/D
        .with_columns(
            pl.col("items").list.eval(
                pl.struct([
                    pl.element().struct.field("courseConjunction").alias("conj"),
                    pl.element().struct.field("items").list.eval(
                        pl.element().struct.field("courseIdentifierParentId")
                    ).alias("items")
                ])#.struct.json_encode()  # uncomment to convert struct to json string
            )
        )
        
        # ensure AND groupings are grouped together by the proper group conjunction
        # with OR as the default if articulation exists else null
        .with_columns(
            pl.when(pl.col("items").is_not_null())
            .then(
                    pl.struct([
                    pl.when(
                        pl.col("sendingArticulation")
                        .struct.field("courseGroupConjunctions")
                        .list.len() > 0
                    )
                    .then(
                        pl.col("sendingArticulation")
                        .struct.field("courseGroupConjunctions")
                        .list.first()
                        .struct.field("groupConjunction")
                    )
                    .otherwise(pl.lit("Or"))
                    .alias("groupConj"),
                    
                    pl.col("items")
                ])
            )
            .otherwise(None)
        )
        # drop intermediary columns
        .drop(["series", "courses", "sendingArticulation", "items"])
    ).rename({"groupConj": "articulation"})
    
    return articulations # return the result


def drop_duplicates_by_term(df: pl.DataFrame) -> pl.DataFrame:
    order_terms = {"W": 0, "S": 1, "Su": 2, "F": 3}
    # separate the term from the year
    df = df.with_columns(
        pl.col("begin").str.slice(-4, 4).cast(pl.Int32).alias("year"),
        pl.col("begin").str.slice(0, pl.col("begin").str.len_chars() - 4).alias("term")
    )
    # the higher the value, the newer the agreement
    df = df.with_columns(
        (pl.col("year") * 10 + pl.col("term").replace_strict(order_terms)).alias("combined")
    )  
    # remove all but the newest instance of a course
    df = df.sort("combined", descending=True).unique("course_id", keep="first")
    
    return df.drop(["begin", "term", "year", "combined"])

In [26]:
import os
from collections import defaultdict

errored = defaultdict(list)

def aggregate_agreements() -> pl.DataFrame:
    with open("./data/institutions_cc.json", "r") as cc_fp, open("./data/institutions_state.json", "r") as uni_fp:
        cc_ids = json.load(cc_fp).keys()
        uni_ids = json.load(uni_fp).keys()
    
    articulations = []
    for uni in uni_ids:
        # articulations.extend([
        #     extract_articulations(cc, uni)
        #     for cc in cc_ids
        #     if os.path.exists(f"./data/{uni}/{cc}to{uni}.json")
        # ])
        for cc in cc_ids:
            if not os.path.exists(f"./data/{uni}/{cc}to{uni}.json"):
                continue
            try:
                articulations.append(extract_articulations(int(cc), int(uni)))
            except Exception:
                errored[uni].append(cc)
    with open("data/known_errors_agreements.json", "w") as fp:
        json.dump(errored, fp, indent=2)
    return pl.concat(articulations)

all_agreements = aggregate_agreements().filter(pl.col("articulation").is_not_null())

In [27]:
extract_articulations(45, 7)

course_id,cc,uni,articulation
i64,i32,i32,struct[2]
384566,45,7,null
384567,45,7,null
384569,45,7,null
384570,45,7,null
289392,45,7,"{""Or"",[{""And"",[200913]}]}"
…,…,…,…
294895,45,7,null
381249,45,7,null
381250,45,7,null


In [28]:
all_agreements.filter(pl.col("articulation").is_not_null())

course_id,cc,uni,articulation
i64,i32,i32,struct[2]
258514,2,1,"{""Or"",[{""And"",[273080]}]}"
249817,2,1,"{""Or"",[{""And"",[273082]}]}"
249819,2,1,"{""Or"",[{""And"",[242283]}]}"
365221,2,1,"{""Or"",[{""And"",[280494]}]}"
365222,2,1,"{""Or"",[{""And"",[280494]}]}"
…,…,…,…
356660,200,144,"{""Or"",[{""And"",[377205]}]}"
356661,200,144,"{""Or"",[{""And"",[377200]}]}"
269491,200,144,"{""Or"",[{""And"",[377153]}, {""And"",[377182]}]}"


In [ ]:
filtered_agreements = all_agreements.filter(pl.col("articulation").is_not_null())

unique_classes = (
    filtered_agreements
    .get_column("articulation")
    .struct.field("items")
    .explode()
    .struct.field("items")
    .explode()
).drop_nulls().unique()

# unique_classes.is_in(filtered_agreements.get_column("course_id"))
unique_classes.filter(unique_classes.is_in(filtered_agreements.get_column("course_id")))
filtered_agreements = filtered_agreements.with_columns(
    pl.col("articulation").struct.json_encode()
)

filtered_agreements

course_id,cc,uni,articulation
i64,i32,i32,str
258514,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
249817,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
249819,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
365221,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
365222,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
…,…,…,…
356660,200,144,"""{""groupConj"":""Or"",""items"":[{""c…"
356661,200,144,"""{""groupConj"":""Or"",""items"":[{""c…"
269491,200,144,"""{""groupConj"":""Or"",""items"":[{""c…"


course_id,cc,uni,articulation
i64,i32,i32,str
258514,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
249817,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
249819,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
365221,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
365222,2,1,"""{""groupConj"":""Or"",""items"":[{""c…"
…,…,…,…
356660,200,144,"""{""groupConj"":""Or"",""items"":[{""c…"
356661,200,144,"""{""groupConj"":""Or"",""items"":[{""c…"
269491,200,144,"""{""groupConj"":""Or"",""items"":[{""c…"


In [31]:
"""
Create a test run of a full course glossary
"""

def create_full_glossary():
    with open("./data/institutions_cc.json", "r") as cc_fp, open("./data/institutions_state.json", "r") as uni_fp:
        cc_ids = json.load(cc_fp).keys()
        uni_ids = json.load(uni_fp).keys()
    
    glossary = []
    cc_count = 0
    for uni_id in uni_ids:
        batch = []
        for cc_id in cc_ids:
            try:
                batch.append(create_course_glossary(cc=cc_id,uni=uni_id))
                cc_count += 1
            except FileNotFoundError:  # no query exists, no query was scraped
                continue
        if batch:
            glossary.extend(batch)
        print("created", cc_count, "dfs for uni ID:", uni_id)
        cc_count = 0
    print("glossary list constructed")
    return pl.concat(glossary).unique()

In [32]:
glossary: pl.DataFrame = drop_duplicates_by_term(create_full_glossary().unique())

created 115 dfs for uni ID: 1
created 115 dfs for uni ID: 7
created 115 dfs for uni ID: 11
created 115 dfs for uni ID: 12
created 115 dfs for uni ID: 21
created 0 dfs for uni ID: 23
created 115 dfs for uni ID: 24
created 11 dfs for uni ID: 26
created 90 dfs for uni ID: 29
created 115 dfs for uni ID: 39
created 115 dfs for uni ID: 42
created 115 dfs for uni ID: 46
created 115 dfs for uni ID: 50
created 115 dfs for uni ID: 60
created 115 dfs for uni ID: 75
created 115 dfs for uni ID: 76
created 115 dfs for uni ID: 79
created 115 dfs for uni ID: 81
created 48 dfs for uni ID: 85
created 115 dfs for uni ID: 88
created 114 dfs for uni ID: 89
created 115 dfs for uni ID: 98
created 0 dfs for uni ID: 115
created 115 dfs for uni ID: 116
created 115 dfs for uni ID: 117
created 115 dfs for uni ID: 120
created 115 dfs for uni ID: 128
created 115 dfs for uni ID: 129
created 115 dfs for uni ID: 132
created 113 dfs for uni ID: 141
created 115 dfs for uni ID: 143
created 115 dfs for uni ID: 144
glossar

In [39]:
"""
Dump glossary with SQLite3
"""

import sqlite3 as sql3

with sql3.connect("./data/courses.db") as conn:
    cursor = conn.cursor()
    
    # create table
    cursor.execute("DROP TABLE IF EXISTS course_glossary;")
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS course_glossary (
            course_id INTEGER PRIMARY KEY,
            inst_id INTEGER,
            course_code TEXT,
            course_name TEXT,
            min_units REAL,
            max_units REAL,
            begin TEXT
        );
    """)
    
    pd.DataFrame(glossary, columns=glossary.columns).to_sql(
        name="course_glossary",
        if_exists="append",
        con=conn,
        index=False
    )
    
with sql3.connect("./data/articulations.db") as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS articulations;")
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articulations (
            course_id INTEGER,
            cc INTEGER,
            uni INTEGER,
            articulation TEXT
        );
    """)
    
    pd.DataFrame(filtered_agreements, columns=filtered_agreements.columns).to_sql(
        name="articulations",
        if_exists="append",
        con=conn,
        index=False
    )